In [3]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

attack_mapping = {
    "Benign": 0,
    "scanning": 1,
    "xss": 2,
    "ddos": 3,
    "dos": 4,
    "injection": 5,
    "password": 6,
    "mitm": 7,
    "ransomware": 8,
    "backdoor": 9,
}

attack_mapping_reversed = {v: k for k, v in attack_mapping.items()}

models = [
    "cross_entropy_acc/cross_entropy_acc_focused", 
    "cross_entropy_balanced_acc/cross_entropy_balanced_acc_focused", 
    "weighted_cross_entropy_acc/weighted_cross_entropy_acc_focused",
    "weighted_cross_entropy_balanced_acc/weighted_cross_entropy_balanced_acc_focused",
    "focal_loss_acc/focal_loss_acc_focused", 
    "focal_loss_balanced_acc/focal_loss_balanced_acc_focused", 
]
sizes = [8, 16, 32, 64]

y_test = pd.read_parquet('../dask_output/rmv_outlier_std_801010/y_test_attack/')
y_test = y_test["Attack"]

y_test_binary = y_test.apply(lambda x: 0 if x == 0 else 1)

results = []

for model in models:
    for size in sizes:
        file_path = f"{model}_y_pred_size_{size}.csv"

        y_pred = pd.read_csv(file_path)["y_pred"]

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average="macro")
        class_f1_scores = f1_score(y_test, y_pred, average=None)

        y_pred_binary = y_pred.apply(lambda x: 0 if x == 0 else 1)
        binary_acc = accuracy_score(y_test_binary, y_pred_binary)

        base_model_name = model.split('/')[-1].replace('_focused', '').replace('_', ' ')
        early_stopping = "Accuracy" if "acc" in base_model_name and not "balanced" in base_model_name else "Balanced Accuracy"
        tabnet_model_name = f"TabNet {base_model_name.title()}"

        results.append([
            tabnet_model_name, early_stopping, size, acc, f1, binary_acc, *class_f1_scores
        ])

columns = [
    "Model Name", "Early Stopping", "Size", "Accuracy", "F1 Score", "Binary Accuracy"
] + [f"{attack_mapping_reversed[i]} F1" for i in range(len(attack_mapping))]
results_df = pd.DataFrame(results, columns=columns)

results_df.to_excel("model_metrics_results.xlsx", index=False)


In [4]:
results_df

,Model Name,Early Stopping,Size,Accuracy,F1 Score,Binary Accuracy,Benign F1,scanning F1,xss F1,ddos F1,dos F1,injection F1,password F1,mitm F1,ransomware F1,backdoor F1
0,TabNet Cross Entropy Acc,Accuracy,8,0.982348,0.693059,0.993069,0.991971,0.990588,0.979804,0.967837,0.977333,0.739494,0.916900,0.000000,0.366667,0.000000
1,TabNet Cross Entropy Acc,Accuracy,16,0.986126,0.851083,0.994485,0.993621,0.992803,0.980948,0.975874,0.977517,0.815778,0.934817,0.138298,0.886364,0.814815
2,TabNet Cross Entropy Acc,Accuracy,32,0.981566,0.718376,0.992148,0.990903,0.989345,0.978528,0.969473,0.977235,0.744252,0.910652,0.000000,0.623377,0.000000
3,TabNet Cross Entropy Acc,Accuracy,64,0.983841,0.818348,0.994184,0.993280,0.992541,0.980587,0.968704,0.977509,0.746888,0.919074,0.091912,0.727273,0.785714
4,TabNet Cross Entropy Balanced Acc,Balanced Accuracy,8,0.980495,0.709671,0.993659,0.992662,0.991641,0.980085,0.963495,0.977383,0.633015,0.860920,0.065934,0.631579,0.000000
5,TabNet Cross Entropy Balanced Acc,Balanced Accuracy,16,0.984015,0.831012,0.994265,0.993369,0.992573,0.980598,0.968458,0.977475,0.746810,0.929039,0.145078,0.761905,0.814815
6,TabNet Cross Entropy Balanced Acc,Balanced Accuracy,32,0.982096,0.824034,0.994099,0.993175,0.992232,0.980342,0.964575,0.977524,0.691520,0.888882,0.142096,0.795181,0.814815
7,TabNet Cross Entropy Balanced Acc,Balanced Accuracy,64,0.983005,0.829176,0.993415,0.992386,0.991415,0.980130,0.968612,0.977524,0.738765,0.917433,0.135652,0.804124,0.785714
8,TabNet Weighted Cross Entropy Acc,Accuracy,8,0.948793,0.669298,0.987411,0.985352,0.981572,0.967370,0.958583,0.673004,0.653929,0.899584,0.030842,0.300319,0.242424
9,TabNet Weighted Cross Entropy Acc,Accuracy,16,0.940568,0.607122,0.986016,0.983704,0.980387,0.966458,0.957723,0.674989,0.340410,0.685198,0.030689,0.256545,0.195122
